In [15]:
import os
import sys
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pandas as pd
import torchvision
from torchvision.io import read_image
from torch.utils.data import Dataset
from torchvision.transforms import ToTensor
from torchvision import datasets
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
import numpy as np
from PIL import Image

In [16]:
# code for hpc:
# labels_path = '/groups/CS156b/data/student_labels/train2023.csv'
# img_dir = '/groups/CS156b/data/train'

# code for local:

# Training  
labels_path_train = 'data/train/labels/labels.csv'
img_dir_train = 'data/train/images'

# Test 
labels_path_test = 'data/test/ids.csv'
img_dir_test = 'data/test/images'

df_train = pd.read_csv(labels_path_train)[:-1]
display(df_train.head())

df_test = pd.read_csv(labels_path_test)[:-1]
display(df_test.head())

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,Path,Sex,Age,Frontal/Lateral,AP/PA,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,Pneumonia,Pleural Effusion,Pleural Other,Fracture,Support Devices
0,0,0,0,0,0,train/pid50512/study1/view1_frontal.jpg,Female,68,Frontal,AP,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1,1,1,1,1,1,train/pid21580/study2/view1_frontal.jpg,Female,87,Frontal,AP,-1.0,NaN,0.0,1.0,NaN,0.0,NaN,1.0,NaN
2,2,2,2,2,2,train/pid21580/study1/view1_frontal.jpg,Female,83,Frontal,AP,-1.0,NaN,NaN,1.0,NaN,NaN,NaN,1.0,NaN
3,3,3,3,3,3,train/pid21580/study1/view2_lateral.jpg,Female,83,Lateral,NaN,-1.0,NaN,NaN,1.0,NaN,NaN,NaN,1.0,NaN
4,4,4,4,4,4,train/pid33839/study1/view1_frontal.jpg,Male,41,Frontal,AP,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,Unnamed: 0,Unnamed: 0.1,Id,Path
0,0,0,18,test/pid56785/study1/view1_frontal.jpg
1,1,1,19,test/pid56785/study1/view2_lateral.jpg
2,2,2,44,test/pid57943/study1/view1_frontal.jpg
3,3,3,45,test/pid57943/study2/view1_frontal.jpg
4,4,4,57,test/pid54805/study1/view1_frontal.jpg


In [17]:
class CustomImageDataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
        self.img_labels = pd.read_csv(annotations_file)[:-1]
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        row = self.img_labels.iloc[idx]

        img_path = row['Path'].split('/')
        img_path = '/'.join(img_path[1:])
        img_path = os.path.join(self.img_dir, img_path)

        # image = read_image(img_path)
        image = Image.open(img_path) # PIL image for applying transform for pre-trained ResNet model 
        image = Image.fromarray(np.stack((image,)*3, axis=-1)) # convert to RGB
        label = list(row[-9:]) # extract label, the last 9 columns

        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)

        return image, label

In [18]:
# Transform for ResNet 
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

training_data = CustomImageDataset(labels_path_train, img_dir_train, transform=transform)
test_data = CustomImageDataset(labels_path_test, img_dir_test, transform=transform)
a, b = training_data[0]
for a, b in training_data:
    print(b)

[1.0, nan, nan, nan, nan, nan, nan, nan, 1.0]
[-1.0, nan, 0.0, 1.0, nan, 0.0, nan, 1.0, nan]
[-1.0, nan, nan, 1.0, nan, nan, nan, 1.0, nan]
[-1.0, nan, nan, 1.0, nan, nan, nan, 1.0, nan]
[-1.0, nan, nan, nan, nan, nan, nan, nan, nan]
[1.0, -1.0, nan, nan, nan, -1.0, nan, nan, nan]
[1.0, -1.0, nan, nan, nan, -1.0, nan, nan, nan]
[1.0, -1.0, nan, nan, nan, -1.0, nan, nan, nan]
[-1.0, nan, 1.0, 1.0, nan, -1.0, nan, -1.0, 1.0]
[-1.0, 1.0, nan, 1.0, nan, -1.0, nan, -1.0, 1.0]
[-1.0, nan, nan, 1.0, nan, 1.0, nan, nan, 1.0]
[-1.0, nan, nan, 1.0, nan, 1.0, nan, nan, 1.0]
[-1.0, nan, 1.0, nan, nan, nan, nan, nan, nan]
[-1.0, nan, 1.0, nan, nan, nan, nan, nan, nan]
[0.0, -1.0, nan, nan, nan, -1.0, nan, nan, nan]
[0.0, -1.0, nan, nan, nan, -1.0, nan, nan, nan]
[-1.0, nan, nan, 1.0, nan, 0.0, nan, nan, 1.0]
[-1.0, nan, nan, 1.0, nan, nan, nan, nan, 1.0]
[-1.0, nan, nan, 1.0, nan, nan, nan, nan, 1.0]
[-1.0, nan, nan, 1.0, nan, 0.0, nan, nan, 1.0]
[-1.0, nan, nan, 1.0, nan, 1.0, nan, nan, 1.0]
[-1.0

In [19]:
train_dataloader = DataLoader(training_data, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=64, shuffle=True)
for x, y in train_dataloader:
    print(x)
    break

tensor([[[[ 0.4679, -0.1314,  0.8789,  ..., -0.0116, -0.0629, -0.1143],
          [ 0.5022, -0.6281, -0.6452,  ..., -0.1486, -0.1828, -0.2513],
          [ 0.3309,  0.2282,  0.4337,  ..., -0.1314, -0.1828, -0.3369],
          ...,
          [-1.8268, -1.7754, -1.7412,  ...,  2.0605,  2.0605,  2.0263],
          [-1.8610, -1.7754, -1.7412,  ...,  2.0948,  2.0948,  2.0777],
          [-1.8610, -1.7925, -1.7412,  ...,  2.1119,  2.0948,  2.0948]],

         [[ 0.6078, -0.0049,  1.0280,  ...,  0.1176,  0.0651,  0.0126],
          [ 0.6429, -0.5126, -0.5301,  ..., -0.0224, -0.0574, -0.1275],
          [ 0.4678,  0.3627,  0.5728,  ..., -0.0049, -0.0574, -0.2150],
          ...,
          [-1.7381, -1.6856, -1.6506,  ...,  2.2360,  2.2360,  2.2010],
          [-1.7731, -1.6856, -1.6506,  ...,  2.2710,  2.2710,  2.2535],
          [-1.7731, -1.7031, -1.6506,  ...,  2.2885,  2.2710,  2.2710]],

         [[ 0.8274,  0.2173,  1.2457,  ...,  0.3393,  0.2871,  0.2348],
          [ 0.8622, -0.2881, -

In [86]:
classes = ['No Finding', 'Enlarged Cardiomediastinum', 'Cardiomegaly', 'Lung Opacity', 'Pneumonia', 'Pleural Effusion', 'Pleural Other', 'Fracture', 'Support Devices']

In [93]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)

if torch.cuda.is_available():
    model.to_cuda()

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=0.0001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor = 0.1, patience=5)

Using cache found in /Users/oliviaxu/.cache/torch/hub/pytorch_vision_v0.10.0
/Users/oliviaxu/anaconda3/envs/myenv/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/oliviaxu/anaconda3/envs/myenv/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
n_epochs = 200

for epoch in range(n_epochs):
    losses = []
    running_loss = 0
    for i, inp in enumerate(train_dataloader):
        inputs, labels = inp
        inputs, labels = inputs.to('cuda'), labels.to('cuda')
        optimizer.zero_grad()
    
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        losses.append(loss.item())

        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        if i%100 == 0 and i > 0:
            print(f'Loss [{epoch+1}, {i}](epoch, minibatch): ', running_loss / 100)
            running_loss = 0.0

    avg_loss = sum(losses)/len(losses)
    scheduler.step(avg_loss)
            
print('Training Done')